# ANPR Archivio storico dei comuni

Data processing che partendo dal dataset di [ANPR](https://www.anpr.interno.it/portale/) contenente l'Archivio storico dei comuni (02-04-2019) - xls predispone e arricchisce il dataset in modo che possa essere importato in DAF e  interoperabile con il dataset rilasciato da ISTAT.

Il dataset verrà poi utilizzato come dataset master anche per l'arricchimento delle informazioni ad oggi contenute in IPA oltre ad essere collegato a diverse grafi come ISPRA, DBPedia...

Su DAF è presente il dataset originario creato da Alberto
https://dataportal.daf.teamdigitale.it/#/private/dataset/anpr_comuni_italiani

Sul dataset pubblicato da ANPR abbiamo aperto degli issue richiedendo che il dataset venga propriamente pubblicato seguendo le linee guida

- https://github.com/italia/anpr/issues/1252
- https://github.com/italia/anpr/issues/1253
 

Operazioni da fare pre processing:

- scraper dataset: non c'è una permament uri
- utilizzo encoding corretto
- attenzione a campi null
- il codice istat deve essere letto come string 
- campo IDPROVINCIA deve essere su tre char con padding a 0 per poter fare join con ISTAT
- formattazione campi data
- formattazione campi nome comune per denominazione in altra lingua 
- formattazione campi nome comune it e denominazione translitterata

Post processing:
- script SQL per estrarre i blocchi di comuni con denominazione in DE e FR per interlinking con Giorgia


In [143]:
import pandas as pd
import numpy as np

In [144]:
# default data per comuni validi
data_cessazione = '9999-12-31'

In [145]:
#fare attenzione a codice istat che deve essere letto come stringa per non perdere gli 0 in testa
#na_filter=False per non inserire NaN ma "" dove i campi sono vuoti

In [146]:
anpr = pd.read_excel("./data/tabella 3 archivio comuni 20190402.xlsx",  dtype={'CODISTAT': str}, na_filter=False)

In [147]:
anpr.dtypes

ID                          int64
DATAISTITUZIONE            object
DATACESSAZIONE             object
CODISTAT                   object
CODCATASTALE               object
DENOMINAZIONE_IT           object
DENOMTRASLITTERATA         object
ALTRADENOMINAZIONE         object
ALTRADENOMTRASLITTERATA    object
IDPROVINCIA                 int64
IDREGIONE                  object
IDPREFETTURA               object
STATO                      object
SIGLAPROVINCIA             object
FONTE                      object
DATAULTIMOAGG              object
COD_DENOM                  object
dtype: object

In [148]:
# il campo provincia è numerico ma su dataset è presentato in formato alfanumerico quindi bisogna trasformarlo
# in string e fare un padding left

### Check campi null

In [150]:
anpr.isnull().sum()

ID                         0
DATAISTITUZIONE            0
DATACESSAZIONE             0
CODISTAT                   0
CODCATASTALE               0
DENOMINAZIONE_IT           0
DENOMTRASLITTERATA         0
ALTRADENOMINAZIONE         0
ALTRADENOMTRASLITTERATA    0
IDPROVINCIA                0
IDREGIONE                  0
IDPREFETTURA               0
STATO                      0
SIGLAPROVINCIA             0
FONTE                      0
DATAULTIMOAGG              0
COD_DENOM                  0
dtype: int64

In [151]:
anpr.head(100)

,ID,DATAISTITUZIONE,DATACESSAZIONE,CODISTAT,CODCATASTALE,DENOMINAZIONE_IT,DENOMTRASLITTERATA,ALTRADENOMINAZIONE,ALTRADENOMTRASLITTERATA,IDPROVINCIA,IDREGIONE,IDPREFETTURA,STATO,SIGLAPROVINCIA,FONTE,DATAULTIMOAGG,COD_DENOM
0,12560,19/11/1866,13/11/1924,028001,A001,ABANO,ABANO,,,28,05,,C,PD,,17/06/2016,028500
1,1,14/11/1924,31/12/9999,028001,A001,ABANO TERME,ABANO TERME,,,28,05,PD,A,PD,,17/06/2016,
2,3,01/04/1864,05/12/1928,001801,A003,ABBADIA ALPINA,ABBADIA ALPINA,,,1,01,,C,TO,,11/11/2017,
3,5,17/03/1861,15/04/1992,015001,A004,ABBADIA CERRETO,ABBADIA CERRETO,,,15,03,,C,MI,,17/06/2016,
4,4,16/04/1992,31/12/9999,098001,A004,ABBADIA CERRETO,ABBADIA CERRETO,,,98,03,LO,A,LO,,17/06/2016,
5,6,06/03/1928,15/04/1992,013001,A005,ABBADIA LARIANA,ABBADIA LARIANA,,,13,03,,C,CO,,17/06/2016,
6,7,16/04/1992,31/12/9999,097001,A005,ABBADIA LARIANA,ABBADIA LARIANA,,,97,03,LC,A,LC,,17/06/2016,
7,8,22/07/1867,31/12/9999,052001,A006,ABBADIA SAN SALVATORE,ABBADIA SAN SALVATORE,,,52,09,SI,A,SI,,17/06/2016,
8,2,23/02/1863,06/03/1928,013801,A002,ABBADIA SOPRA ADDA,ABBADIA SOPRA ADDA,,,13,03,,C,CO,,17/06/2016,
9,11,17/03/1861,27/07/1927,092001,A007,ABBASANTA,ABBASANTA,,,92,20,,C,CA,,17/06/2016,


## Adattamento campo IDPROVINCIA 

Leggerlo come striga e fare padding con 0 su tre char

In [152]:
anpr['IDPROVINCIA'] = anpr['IDPROVINCIA'].apply(str)

In [153]:
anpr['IDPROVINCIA'] = anpr['IDPROVINCIA'].apply(lambda x: x.zfill(3))

## Adattamento formato campi data

I campi data, che al momento non sono gestiti sul form di ingestion del DAF, vanno trasformati usando il "-" come separatore.

Da fare per il dataset nuovo per il cui non sarà più necessario fare vista.

In [154]:
anpr['DATAULTIMOAGG'] = pd.to_datetime(anpr['DATAULTIMOAGG'])
anpr['DATAULTIMOAGG'] = anpr['DATAULTIMOAGG'].dt.strftime('%Y-%m-%d')

In [155]:
#su data ultimo agg bisogna ignorare errori
anpr['DATAISTITUZIONE'] = pd.to_datetime(anpr['DATAISTITUZIONE'])
anpr['DATAISTITUZIONE'] = anpr['DATAISTITUZIONE'].dt.strftime('%Y-%m-%d')

In [156]:
anpr['DATACESSAZIONE'] = pd.to_datetime(anpr['DATACESSAZIONE'], errors='coerce')

In [135]:
anpr['DATACESSAZIONE'] = anpr['DATACESSAZIONE'].dt.strftime('%Y-%m-%d')

In [157]:
#NON FATTO
#anpr['DATACESSAZIONE'].date.dt.strftime('%Y-%m-%d').replace('NaT', data_cessazione)

In [158]:
anpr['DATACESSAZIONE'] = anpr['DATACESSAZIONE'].replace('NaT', data_cessazione)

## Adattamento 
###  DENOMINAZIONE_IT e DENOMTRASLITTERATA

1) Tutte le denominazioni in italiano **translitterate** le dobbiamo trasformare in LetteraMaiuscolaLettereminuscole
così diventa confrontabile con i comuni di IPA

2) Tutte le denominazioni in italiano (non translitterate) le dobbiamo trasformare in LetteraMaiuscolaLettereminuscole. 

In presenza di doppia parola (es ABANO TERME) dobbiamo farla diventare LetteraMaiuscolaLettereminuscole_LetteraMaiuscolalettereminuscole  
queste due cose sono fondamentali anche per diversi join sui dataset IPA e per fare intelinking con DBPedia.

In [159]:
anpr['result'] = np.where(anpr['DENOMINAZIONE_IT'] == anpr['DENOMTRASLITTERATA'], 'no change', 'changed')
print (anpr.result.value_counts())

no change    18890
changed        413
Name: result, dtype: int64


In [160]:
anpr[anpr['result']=='changed']

,ID,DATAISTITUZIONE,DATACESSAZIONE,CODISTAT,CODCATASTALE,DENOMINAZIONE_IT,DENOMTRASLITTERATA,ALTRADENOMINAZIONE,ALTRADENOMTRASLITTERATA,IDPROVINCIA,IDREGIONE,IDPREFETTURA,STATO,SIGLAPROVINCIA,FONTE,DATAULTIMOAGG,COD_DENOM,result
93,17704,1861-03-17,1929-08-22,087006,A056,ADERNÒ,ADERNO',,,087,19,,C,CT,,2016-06-17,087500,changed
126,17720,1861-03-17,1927-11-01,001001,A074,AGLIÈ,AGLIE',,,001,01,,C,TO,,2016-06-17,,changed
127,22003,1927-12-01,1929-06-14,007805,A074,AGLIÈ,AGLIE',,,007,02,,C,AO,,2016-06-17,,changed
128,22004,1929-06-15,1945-04-10,007805,A074,AGLIÈ,AGLIE',,,007,02,,C,AO,,2016-06-17,,changed
129,17718,1945-05-10,1947-09-01,001001,A074,AGLIÈ,AGLIE',,,001,01,,C,TO,,2016-06-17,,changed
130,17719,1947-10-01,1947-01-23,001001,A074,AGLIÈ,AGLIE',,,001,01,,C,TO,,2016-06-17,,changed
131,82,1947-01-24,NaT,001001,A074,AGLIÈ,AGLIE',,,001,01,TO,A,TO,,2016-06-17,,changed
210,130,1864-01-14,2005-12-31,090002,A115,ALÀ DEI SARDI,ALA' DEI SARDI,,,090,20,,C,SS,,2016-06-17,,changed
211,129,2006-01-01,2016-04-27,104003,A115,ALÀ DEI SARDI,ALA' DEI SARDI,,,104,20,,C,OT,,2017-11-11,,changed
212,22823,2016-04-28,NaT,090002,A115,ALÀ DEI SARDI,ALA' DEI SARDI,,,090,20,SS,A,SS,,2017-11-11,,changed


In [161]:
anpr["Denominazione_it_linking"] = anpr.DENOMINAZIONE_IT.str.title()
anpr["Denominazione_it_linking"] = anpr.Denominazione_it_linking.str.replace(" ", "_")

In [162]:
anpr["DENOMTRASLITTERATA_IPA"]=anpr.DENOMTRASLITTERATA.str.title()

In [163]:
anpr

,ID,DATAISTITUZIONE,DATACESSAZIONE,CODISTAT,CODCATASTALE,DENOMINAZIONE_IT,DENOMTRASLITTERATA,ALTRADENOMINAZIONE,ALTRADENOMTRASLITTERATA,IDPROVINCIA,IDREGIONE,IDPREFETTURA,STATO,SIGLAPROVINCIA,FONTE,DATAULTIMOAGG,COD_DENOM,result,Denominazione_it_linking,DENOMTRASLITTERATA_IPA
0,12560,1866-11-19,1924-11-13,028001,A001,ABANO,ABANO,,,028,05,,C,PD,,2016-06-17,028500,no change,Abano,Abano
1,1,1924-11-14,NaT,028001,A001,ABANO TERME,ABANO TERME,,,028,05,PD,A,PD,,2016-06-17,,no change,Abano_Terme,Abano Terme
2,3,1864-01-04,1928-05-12,001801,A003,ABBADIA ALPINA,ABBADIA ALPINA,,,001,01,,C,TO,,2017-11-11,,no change,Abbadia_Alpina,Abbadia Alpina
3,5,1861-03-17,1992-04-15,015001,A004,ABBADIA CERRETO,ABBADIA CERRETO,,,015,03,,C,MI,,2016-06-17,,no change,Abbadia_Cerreto,Abbadia Cerreto
4,4,1992-04-16,NaT,098001,A004,ABBADIA CERRETO,ABBADIA CERRETO,,,098,03,LO,A,LO,,2016-06-17,,no change,Abbadia_Cerreto,Abbadia Cerreto
5,6,1928-06-03,1992-04-15,013001,A005,ABBADIA LARIANA,ABBADIA LARIANA,,,013,03,,C,CO,,2016-06-17,,no change,Abbadia_Lariana,Abbadia Lariana
6,7,1992-04-16,NaT,097001,A005,ABBADIA LARIANA,ABBADIA LARIANA,,,097,03,LC,A,LC,,2016-06-17,,no change,Abbadia_Lariana,Abbadia Lariana
7,8,1867-07-22,NaT,052001,A006,ABBADIA SAN SALVATORE,ABBADIA SAN SALVATORE,,,052,09,SI,A,SI,,2016-06-17,,no change,Abbadia_San_Salvatore,Abbadia San Salvatore
8,2,1863-02-23,1928-06-03,013801,A002,ABBADIA SOPRA ADDA,ABBADIA SOPRA ADDA,,,013,03,,C,CO,,2016-06-17,,no change,Abbadia_Sopra_Adda,Abbadia Sopra Adda
9,11,1861-03-17,1927-07-27,092001,A007,ABBASANTA,ABBASANTA,,,092,20,,C,CA,,2016-06-17,,no change,Abbasanta,Abbasanta


In [164]:
anpr.DATACESSAZIONE.value_counts()

1927-11-01    1632
1923-04-02     589
1992-04-15     368
1923-07-02     256
1968-05-04     187
1974-08-19     177
2016-04-27     155
1923-04-26     142
1970-02-03     135
2005-12-31     123
1947-09-15     118
1935-07-04     105
1945-08-31      79
1923-12-21      78
1992-05-22      77
1923-03-18      68
2015-12-31      68
1998-03-09      67
1945-04-10      63
1928-11-05      56
2009-06-29      50
1928-12-05      46
2018-12-31      45
1928-05-09      45
1928-01-25      43
2009-11-07      40
1928-12-03      40
1946-01-29      39
1928-02-20      38
1928-04-16      37
              ... 
1928-08-17       1
1930-11-17       1
1961-02-10       1
1991-06-17       1
1930-09-13       1
1919-06-04       1
1902-09-20       1
1930-06-29       1
1957-08-13       1
1941-08-19       1
1916-09-11       1
1937-10-15       1
2001-11-12       1
1950-06-28       1
1919-08-31       1
1975-04-23       1
1944-07-03       1
1953-05-12       1
1910-05-01       1
1927-04-23       1
1966-10-14       1
1911-04-07  

## Salvare dataset finale con separatore e senza index

In [165]:
anpr.to_csv("./data/output.csv", index=False, sep=";")